Para importar as bibliotecas

In [20]:
import pandas as pd
import qrcode
import os

In [22]:
base_html= pd.read_excel('./CursoPython.xlsx', sheet_name= "Candidatos", usecols=['Nome','Turma', 'Sala', 'Professor', 'Inscrição', 'Dia', 'Notebook'])


In [23]:
base_html

,Inscrição,Nome,Notebook,Turma,Professor,Sala,Dia
0,9EADL1IG0S,Agatha Milleny Moreira de Souza,SIM,Turma16,Myrna,S13,Quinta 1
1,XHGE0817LP,Beatriz Falbo Bernardo,SIM,Turma15,Alex,S12,Quinta 1
2,5R24A0LKEY,Barbara de Paula Silva Gomes,NÃO,Turma13,Modolo,S10,Quinta 1
3,P8HMEB677Z,Lívia dos Praseres Santos,NÃO,Turma11,Grilo,S08,Quinta 1
4,MSHVDDQACW,Amanda Eloisa Santos da Costa,NÃO,Turma11,Grilo,S08,Quinta 1
...,...,...,...,...,...,...,...
749,STVS3G5AO5,Dyana Abílio da Silva,NÃO,Turma9,Nisflei,S11,Segunda 3
750,KCIRCUSMUL,Pedro Henrique de Sena Medeiros,NÃO,Turma9,Nisflei,S11,Segunda 3
751,NLJVLOHJTX,Isabella Vilalba dos Santos,SIM,Turma9,Nisflei,S11,Segunda 3
752,A39SH8PF1H,GUILHERME CSIPOK FEDIZKo,SIM,Turma6,Grilo,S08,Segunda 3


In [24]:
base_html[base_html['Inscrição']=="6VJK0EPF6X"]

,Inscrição,Nome,Notebook,Turma,Professor,Sala,Dia
485,6VJK0EPF6X,Vitor Masao Valfogo Mine,SIM,Turma23,Grilo,S08,Quinta 3


Para gerar as pastas e os páginas HTML

In [25]:
def arquivo_existe(pasta, nome_arquivo_base):
    # Verificar arquivos com extensão .jpg
    caminho_arquivo_jpg = os.path.join(pasta, nome_arquivo_base + '.jpg')
    if os.path.isfile(caminho_arquivo_jpg):
        return pasta + "/" + nome_arquivo_base + ".jpg"
    
    # # Verificar arquivos com extensão .pdf
    # caminho_arquivo_pdf = os.path.join(pasta, nome_arquivo_base + '.pdf')
    # if os.path.isfile(caminho_arquivo_pdf):
    #     return pasta + "/" + nome_arquivo_base + ".pdf"
    
    # Nenhum arquivo encontrado
    return "false"



# for i in range(len(base_html)):
for i in range(0, len(base_html)):
    
    pasta = "./assets"
    nome_arquivo = base_html["Inscrição"][i]
    
    
    retorno = arquivo_existe(pasta, nome_arquivo)
    if retorno == "false":
            site = f"""
            <!DOCTYPE html>
            <html lang="pt-br">
            <head>
                <meta charset="UTF-8">
                <meta name="viewport" content="width=device-width, initial-scale=1.0">
                <title>Germinare TECH</title>
                <style>
                    @import url('https://fonts.googleapis.com/css2?family=Inter:wght@100..900&display=swap');

                    * {{
                        margin: 0;
                        padding: 0;
                        box-sizing: border-box;
                        font-family: 'Inter', sans-serif;
                        user-select: none;
                    }}

                    body {{
                        background-color: #fdfdfd;
                        color: #333;
                        height: 100vh;
                        display: flex;
                        justify-content: center;
                        align-items: center;
                        padding: 1rem;
                    }}

                    .container {{
                        background-color: #fff;
                        border-radius: 1rem;
                        box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
                        padding: 2rem;
                        max-width: 90%;
                        width: 600px;
                        text-align: center;
                    }}

                    h1 {{
                        font-weight: 600;
                        background-color: #9ed1e7;
                        border-radius: .9rem;
                        padding-block: .5rem;
                        padding-inline: 1rem;
                        color: #fff;
                        margin-bottom: 1.5rem;
                    }}

                    p {{
                        margin-block: 1rem;
                        font-size: 1rem;
                        line-height: 1.5;
                    }}

                    strong {{
                        font-size: 1rem;
                        color: #333;
                    }}

                    img {{
                        max-width: 100%;
                        height: auto;
                        margin: 1rem 0;
                    }}
                    
                    .no-image {{
                        font-size: 1rem;
                        color: #ff0000;
                        margin: 1rem 0;
                        font-weight: bold;
                    }}

                    @media only screen and (max-width: 600px) {{
                        h1 {{
                            font-size: 1.5rem;
                        }}

                        p {{
                            font-size: 1rem;
                        }}
                    }}

                    @media only screen and (max-width: 400px) {{
                        h1 {{
                            font-size: 1.2rem;
                        }}

                        p {{
                            font-size: 1rem;
                        }}
                    }}
                </style>
            </head>
            <body>
                <div class="container">
                    <h1>Informações do Candidato</h1>
                    <p id="no-image" class="no-image">Imagem não disponível</p>
                    <p>Inscrição: <strong>{base_html["Inscrição"][i]}</strong></p>
                    <p>Nome: <strong>{base_html["Nome"][i]}</strong></p>
                    <p>Turma: <strong>{base_html["Turma"][i]}</strong></p>
                    <p>Sala: <strong>{base_html["Sala"][i]}</strong></p>
                    <p>Professor: <strong>{base_html["Professor"][i]}</strong></p>
                    <p>Dia e turno: <strong>{base_html["Dia"][i]}</strong></p>
                    <p id="notebook">Notebook: <strong>{base_html["Notebook"][i]}</strong></p>
                </div>

                <script>
                    const notebook = document.getElementById('notebook');
                    if (notebook && notebook.innerText.toLowerCase().includes('sim')) {{
                        notebook.style.color = 'green';
                    }} else if (notebook) {{
                        notebook.style.color = 'red';
                    }}
                </script>
            </body>
            </html>
            """
    else:
        site = f"""
        <!DOCTYPE html>
        <html lang="pt-br">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Germinare TECH</title>
            <style>
                @import url('https://fonts.googleapis.com/css2?family=Inter:wght@100..900&display=swap');

                * {{
                    margin: 0;
                    padding: 0;
                    box-sizing: border-box;
                    font-family: 'Inter', sans-serif;
                    user-select: none;
                }}

                body {{
                    background-color: #fdfdfd;
                    color: #333;
                    height: 100vh;
                    display: flex;
                    justify-content: center;
                    align-items: center;
                    padding: 1rem;
                }}

                .container {{
                    background-color: #fff;
                    border-radius: 1rem;
                    box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
                    padding: 2rem;
                    max-width: 90%;
                    width: 600px;
                    text-align: center;
                }}

                h1 {{
                    font-weight: 600;
                    background-color: #9ed1e7;
                    border-radius: .9rem;
                    padding-block: .5rem;
                    padding-inline: 1rem;
                    color: #fff;
                    margin-bottom: 1.5rem;
                }}

                p {{
                    margin-block: 1rem;
                    font-size: 1rem;
                    line-height: 1.5;
                }}

                strong {{
                    font-size: 1rem;
                    color: #333;
                }}

                img {{
                    max-width: 100%;
                    max-height: 22rem;
                    margin: 1rem 0;
                }}

                @media only screen and (max-width: 600px) {{
                    h1 {{
                        font-size: 1.5rem;
                    }}

                    p {{
                        font-size: 1rem;
                    }}
                }}

                @media only screen and (max-width: 400px) {{
                    h1 {{
                        font-size: 1.2rem;
                    }}

                    p {{
                        font-size: 1rem;
                    }}
                }}
            </style>
        </head>
        <body>
            <div class="container">
                <h1>Informações do Candidato</h1>
                <img src=../../{retorno} alt="{base_html["Inscrição"][i]}">
                <p>Inscrição: <strong>{base_html["Inscrição"][i]}</strong></p>
                <p>Nome: <strong>{base_html["Nome"][i]}</strong></p>
                <p>Turma: <strong>{base_html["Turma"][i]}</strong></p>
                <p>Sala: <strong>{base_html["Sala"][i]}</strong></p>
                <p>Professor: <strong>{base_html["Professor"][i]}</strong></p>
                <p>Dia e turno: <strong>{base_html["Dia"][i]}</strong></p>
                <p id="notebook">Notebook: <strong>{base_html["Notebook"][i]}</strong></p>
            </div>

            <script>
                const notebook = document.getElementById('notebook');
                if (notebook && notebook.innerText.toLowerCase().includes('sim')) {{
                    notebook.style.color = 'green';
                }} else if (notebook) {{
                    notebook.style.color = 'red';
                }}
            </script>
        </body>
        </html>
        """

    # print(info)

    
    folder = os.path.join("paginas_html/", base_html["Inscrição"][i])
    os.mkdir(folder)
    # arquivo = open(f'{folder}/{base_html["Inscrição"][i]}.html', "a")
    with open(f'{folder}/{base_html["Inscrição"][i]}.html', "w", encoding="utf-8") as arquivo:
    #arquivo = open(f'teste.html', "a")

        # print(site)
        arquivo.write(site)
        arquivo.close()

Para gerar os QR_Codes

In [ ]:
df_qr = pd.read_excel('./CursoPython.xlsx', sheet_name="Candidatos",
                              usecols=['Nome','Turma', 'Sala', 'Professor', 'Inscrição','Dia'])

# for i in range(len(df_qr)):
for i in range(0, len(df_qr)):
    nome_insc = ""
    link = f'https://gedaijef.github.io/qrcode/paginas_html/{df_qr["Inscrição"][i].replace(" ", "_")}/{df_qr["Inscrição"][i].replace(" ", "_")}.html'
    qrimg = qrcode.make(link)
    qrimg.save(f'qrCodes/{df_qr["Inscrição"][i].replace(" ", "_")}.png')